#### Build a word-level model Seq2Seq Language Translation model English - Hindi

#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model


#### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Load and Preprocess Data

In [3]:
lines= pd.read_table('drive/My Drive/hin.txt', names=['eng', 'hin', 'na'])
lines = lines.drop(columns = ['na'])

In [4]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.hin=lines.hin.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [5]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of hindi
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [6]:
# Max Length of source sequence
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)
max_length_src



22

In [7]:
# Max Length of target sequence
length_list=[]
for l in lines.hin:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)
max_length_tar

27

In [8]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2367, 2980)

In [9]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens
num_encoder_tokens += 1 
num_encoder_tokens

2368

In [10]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])


In [11]:

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())


In [12]:
lines = shuffle(lines)
lines.head()

,eng,hin
1350,i worked on the farm all day,START_ मैंने पूरे दिन खेत में काम किया। _END
890,hes always dissatisfied,START_ वह कभी भी संतुष्ट नहीं होता। _END
918,i will explain it to him,START_ मैं उसको यह बात समझाउँगा। _END
1186,what happened to our order,START_ हमारे औरडर का क्या हुआ _END
1260,she declined the invitation,START_ उसने आमंत्रण को स्वीकार नहीं किया। _END


#### Train and Validation Split 

In [13]:
# Train - Test Split
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape

((2218,), (555,))

In [14]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [15]:
latent_dim = 256

In [16]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]


#### Create Model 

In [17]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#### Compile Model

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [19]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 150

In [20]:
history = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/150
34/34 [==============================] - 27s 797ms/step - loss: 1.7884 - acc: 0.1346 - val_loss: 1.7337 - val_acc: 0.1421
Epoch 2/150
34/34 [==============================] - 25s 725ms/step - loss: 1.5738 - acc: 0.1515 - val_loss: 1.7057 - val_acc: 0.1450
Epoch 3/150
34/34 [==============================] - 25s 725ms/step - loss: 1.5152 - acc: 0.1548 - val_loss: 1.6872 - val_acc: 0.1579
Epoch 4/150
34/34 [==============================] - 25s 728ms/step - loss: 1.4716 - acc: 0.1639 - val_loss: 1.6899 - val_acc: 0.1547
Epoch 5/150
34/34 [==============================] - 25s 724ms/step - loss: 1.4365 - acc: 0.1776 - val_loss: 1.6681 - val_acc: 0.1747
Epoch 6/150
34/34 [==============================] - 25s 723ms/step - loss: 1.3904 - acc: 0.1926 - val_loss: 1.6677 - val_acc: 0.1767
Epoch 7/150
34/34 [==============================] - 25s 724ms/step - loss: 1.3455 - acc: 0.2018 - val_loss: 1.6526 - va

#### Predictions

In [21]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [22]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [23]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

#### Inference Samples

In [28]:
for dd in range(900,905):
  k+=1
  (input_seq, actual_output), _ = next(val_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input English sentence:', X_test[k:k+1].values[0])
  print('Predicted Hindi Translation:', decoded_sentence[:-4])
  print('\n')

Input English sentence: he is digging his own grave
Predicted Hindi Translation:  वह अपने ओर जैसी है। 


Input English sentence: my sister has long legs
Predicted Hindi Translation:  मेरी बहन मेरी बहन से चुकी है। 


Input English sentence: the law says that all men are equal
Predicted Hindi Translation:  उन्हें सूरज के डूबने से अपनी मंजिल तक पहुँच जाए


Input English sentence: are you busy today
Predicted Hindi Translation:  तुम्हारे पास आज समय है क्या 


Input English sentence: i cant move
Predicted Hindi Translation:  मुझे याद नहीं। 


